# Camera Traps Tutorial for 2020 SnowEx Hackweek
##### Catherine Breen, cbreen@uw.edu
##### Cassie Lumbrazo, lumbraca@uw.edu

# What is a camera trap?

- A camera trap (sometimes referred to as a remote camera or hunting camera) is similar to a normal camera except they:
    - can detect  and take pictures when motion is in their vicinity. 
    - can take pictures at consistent intervals at settings preset by the user.  
    - are designed to withstand harsh conditions and to be left up for long periods of time.  
- Their potential for snow hydrology is exciting. Consistent daily camera trap photos can be analyzed contiously to form a *time-lapse* of snow conditions at one location. They can operate in sub-freezing temperatures and during storms -- conditions when access with other types of measurements would have been difficult. 

# Camera Traps and SnowEx
- Camera traps were installed in both the 2017 and 2020 SnowEx campaigns. 
- During the 2020 SnowEx field campaign, 30 cameras were installed in Grand Mesa, CO between September and May. 
- Cameras were installed across a vegetation gradient on a scale of 1 (least vegetation and ) to 9 (most vegetation). Within each vegetation class there are three sub-classes of snow depths: shallow, intermediate, and deep snow, that were derived from 2017 SnowEx lidar measurements. 
- Below is an example site, site XX, 
- 2017 camera trap time-lapse imagery has been submitted to the NSIDC. 2020 time-lapse is in progress for submission to the NSIDC with estimation of submission as summer 2021. 

## Exercise 1
##### Pulling and visualizing camera trap data

We will now pull all the time-lapse imagery for one camera from the 2020 SnowEx field campaign. This is camera XXX. We will pull all images and display a couple from various times of the winter season. 

In [ ]:
#### pull images from Micah's database

- What do you notice? Is this an open or closed canopy site? 
- What are some snow properties that you might be able to measure using these devices? 

# Applications

##### Extracting snow properties from camera trap images

## Snow Depth

## Exercise 2

In [ ]:
#### pull csv file from Micah's database

## Snow in Trees

Cassie to fill in 

NameError: name 'jb' is not defined